In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
df = pd.read_excel('../Godzilla.xlsx')

# Convert the 'Movie Budget (Yen)' and 'Final Revenue (Yen)' columns to numeric values, removing any non-numeric characters
df['Movie Budget (Yen)'] = df['Movie Budget (Yen)'].replace('[\¥,\,]', '', regex=True).replace('-', np.nan).astype(float)
df['Final Revenue (Yen)'] = df['Final Revenue (Yen)'].replace('[\¥,\,]', '', regex=True).replace('-', np.nan).astype(float)

# Convert Yen to Dollars (1 USD = 140 JPY)
conversion_rate = 140
df['Movie Budget (USD)'] = df['Movie Budget (Yen)'] / conversion_rate
df['Final Revenue (USD)'] = df['Final Revenue (Yen)'] / conversion_rate

# Drop the Yen columns as we are using the USD columns for modeling
df = df.drop(columns=['Movie Budget (Yen)', 'Final Revenue (Yen)'])

df['Movie Budget (USD)'] = df.groupby('Era')['Movie Budget (USD)'].transform(lambda x: x.fillna(x.mean()))

# Select relevant features and target
features = ['Movie Budget (USD)', 'Release Date', 'Era', 'Runtime (mins)', "Godzilla's Sizes (In Meters)"]
target = 'Final Revenue (USD)'

# Extract features and target
X = df[features]
y = df[target]

# Convert 'Release Date' to a numerical format (e.g., year only)
X['Release Year'] = X['Release Date'].dt.year
X = X.drop(columns=['Release Date'])

# Define transformers for numerical and categorical features
numerical_features = ['Movie Budget (USD)', 'Runtime (mins)', "Godzilla's Sizes (In Meters)", 'Release Year']
categorical_features = ['Era']

C:\Users\Lardex\AppData\Local\Temp\ipykernel_22936\882701441.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Release Year'] = X['Release Date'].dt.year


In [28]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Fill missing values with median
    ('scaler', StandardScaler())  # Scale numerical values
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values with most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Encode categorical values
])

# Combine transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),  # Apply numerical transformer to numerical features
        ('cat', categorical_transformer, categorical_features)  # Apply categorical transformer to categorical features
    ])

In [29]:
X_preprocessed = preprocessor.fit_transform(X)

In [30]:
# Split the dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [31]:
# Train the regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the trained model and the preprocessor
# Save the model
with open('godzilla_revenue_model.pkl', 'wb') as f:
    pickle.dump(model, f)
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

In [32]:
# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 318004577942746.9
R-squared: 0.7222010945989945
